# Twirling with Program Sets on mitiq 

In this notebook, we look at Pauli twirling, and specifically how this can be realized using Amazon Braket. 

##

Pauli twirling is a technique used widely across quantum learning and mitigation protocols, known for it's simplicity and effectiveness. Essentially, twirling with classes of channels allows us to control and shape the noise channel. As a result, we can increase confidence in our knowledge of the channel and subsequent mitigation. 

Pauli twirling refers to twirling with elements of the Pauli group (i.e. tensor products of single-qubit Pauli matrices). When acting on a channel, the noise channel becomes diagonalized in the superoperator basis of Pauli matrices.

However, twirled channels themselves are generally not implementable. Instead, we approximate the twirling process using finite sets of twirls. 

Here, we show two ways this can be realized on Braket. The first is to use mitiq's built in Pauli Twirling module. The result is to create many different Paul variants defined for your circuits. 

We can additionally realize these using parameteric compilation. 

In [1]:
from mitiq import pt
from braket.circuits import Circuit 
from braket.circuits.observables import I,X,Y,Z
import numpy as np
import sys
import os 

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir))) # parent directory 
from tools.mitigation_tools import apply_two_qubit_twirl
from braket.circuits.noises import AmplitudeDamping
from braket.circuits.noise_model import NoiseModel, GateCriteria
from braket.circuits.gates import CNot, CZ, Rz
from braket.devices import LocalSimulator
from mitiq.pt import generate_pauli_twirl_variants

from tools.observable_tools import matrix_to_pauli
from mitiq.pt.pt import CZ_twirling_gates, CNOT_twirling_gates, generate_pauli_twirl_variants

from braket.aws import AwsDevice
from braket.program_sets import ProgramSet, CircuitBinding
from qiskit_braket_provider import to_braket, BraketProvider


In [2]:
cnot = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,0,1],
    [0,0,1,0]
])

ps = [I(),X(),Y(),Z()]

for i in ps:
    for j in ps:
        print(
            'inp -> out: ',
            matrix_to_pauli((i @ j).to_matrix())[0][1],'->',
            matrix_to_pauli(cnot @ (i @ j).to_matrix() @ cnot)[0][1])


inp -> out:  II -> II
inp -> out:  IX -> IX
inp -> out:  IY -> ZY
inp -> out:  IZ -> ZZ
inp -> out:  XI -> XX
inp -> out:  XX -> XI
inp -> out:  XY -> YZ
inp -> out:  XZ -> YY
inp -> out:  YI -> YX
inp -> out:  YX -> YI
inp -> out:  YY -> XZ
inp -> out:  YZ -> XY
inp -> out:  ZI -> ZI
inp -> out:  ZX -> ZX
inp -> out:  ZY -> IY
inp -> out:  ZZ -> IZ


### Background on Pauli Twirling Methodologies

For a gate, we are interested in twirling only the set of operations which 

For instance, let $\tilde{\mathcal{E}}_U$ denote the noise application of a unitary channel $U$. Then, we can write this in terms of a before-gate noise $\mathcal{E}_N$ composed with the unitary:

$$\tilde{\mathcal{E}}_{U} =  \mathcal{E}_U \circ \mathcal{E}_N$$

To twirl this, we ideally would apply a Pauli channel as follow:

$ \mathcal{E}_{twirl} = \sum_{\sigma \in \mathbb{P}} \mathcal{E}_U \circ \mathcal{E}_\sigma \circ \mathcal{E}_N \circ \mathcal{E}_\sigma $

However, we can't actually access the noise channel directly, and so we can instead commute (or anti-commute) these Paulis through the CNOT gate. This gives the following input and output Pauli twirls:

| $\sigma_i$ | II  | IX  | IY  | IZ  | XI  | XX  | XY  | XZ  | YI  | YX  | YY  | YZ  | ZI  | ZX  | ZY  | ZZ  |
| ---------- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| $\sigma_o$ | II  | IX  | ZY  | ZZ  | XX  | XI  | YZ  | YY  | YX  | YI  | XZ  | XY  | ZI  | ZX  | IY  | IZ |

Pracitcally, we realize this by sampling the above input and output channel per each CNOT, and placing them before and after each Pauli. The CNOTs operations are preserved up to a global phase.

In [3]:

test = Circuit().cnot(0,1)
variants = generate_pauli_twirl_variants(test,3)
for i in variants:
    print(i)

T  : │  0  │  1  │  2  │
      ┌───┐       ┌───┐ 
q0 : ─┤ X ├───●───┤ Y ├─
      └───┘   │   └───┘ 
      ┌───┐ ┌─┴─┐ ┌───┐ 
q1 : ─┤ Z ├─┤ X ├─┤ Y ├─
      └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │
T  : │  0  │  1  │  2  │
      ┌───┐       ┌───┐ 
q0 : ─┤ Y ├───●───┤ Y ├─
      └───┘   │   └───┘ 
      ┌───┐ ┌─┴─┐ ┌───┐ 
q1 : ─┤ I ├─┤ X ├─┤ X ├─
      └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │
T  : │  0  │  1  │  2  │
      ┌───┐       ┌───┐ 
q0 : ─┤ X ├───●───┤ X ├─
      └───┘   │   └───┘ 
      ┌───┐ ┌─┴─┐ ┌───┐ 
q1 : ─┤ X ├─┤ X ├─┤ I ├─
      └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │


For smaller systems, we can sometime enumerate all potential twirls, or use smaller twirling sets, but as the nubmer of unique circuits scales as $16^{N_{CNOTS}}$, this is quickly intractable. Thus, we just sample different Pauli twirled variants and reconstruct them. Again, these Pauli twirled variants all represent the same logical circuit, and act solely to homegenize the noise channel to a Pauli channel. 

In [54]:

length_bell = 100

test = Circuit().h(0).cnot(0,1)

def ghz_layer():
    return Circuit().cnot(0,1).h(0).h(1).cnot(1,0)

for i in range(length_bell//2):
    test+= ghz_layer()

print(test)

T  : │  0  │  1  │  2  │  3  │  4  │  5  │  6  │  7  │  8  │  9  │ 10  │ 11  │ 12  │ 13  │ 14  │ 15  │ 16  │ 17  │ 18  │ 19  │ 20  │ 21  │ 22  │ 23  │ 24  │ 25  │ 26  │ 27  │ 28  │ 29  │ 30  │ 31  │ 32  │ 33  │ 34  │ 35  │ 36  │ 37  │ 38  │ 39  │ 40  │ 41  │ 42  │ 43  │ 44  │ 45  │ 46  │ 47  │ 48  │ 49  │ 50  │ 51  │ 52  │ 53  │ 54  │ 55  │ 56  │ 57  │ 58  │ 59  │ 60  │ 61  │ 62  │ 63  │ 64  │ 65  │ 66  │ 67  │ 68  │ 69  │ 70  │ 71  │ 72  │ 73  │ 74  │ 75  │ 76  │ 77  │ 78  │ 79  │ 80  │ 81  │ 82  │ 83  │ 84  │ 85  │ 86  │ 87  │ 88  │ 89  │ 90  │ 91  │ 92  │ 93  │ 94  │ 95  │ 96  │ 97  │ 98  │ 99  │ 100 │ 101 │ 102 │ 103 │ 104 │ 105 │ 106 │ 107 │ 108 │ 109 │ 110 │ 111 │ 112 │ 113 │ 114 │ 115 │ 116 │ 117 │ 118 │ 119 │ 120 │ 121 │ 122 │ 123 │ 124 │ 125 │ 126 │ 127 │ 128 │ 129 │ 130 │ 131 │ 132 │ 133 │ 134 │ 135 │ 136 │ 137 │ 138 │ 139 │ 140 │ 141 │ 142 │ 143 │ 144 │ 145 │ 146 │ 147 │ 148 │ 149 │ 150 │ 151 │
      ┌───┐             ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌──

To reduce the compilation overhead, we can envision the following. 

In [55]:

pt_para_circ, pt_params = apply_two_qubit_twirl(test, num_samples=100)

print(pt_para_circ)
print(len(pt_params))

T  : │       0        │       1        │       2        │  3  │       4        │       5        │       6        │       7        │  8  │       9        │       10       │ 11  │       12       │       13       │ 14  │       15       │       16       │       17       │       18       │ 19  │       20       │       21       │ 22  │       23       │       24       │ 25  │       26       │       27       │       28       │       29       │ 30  │       31       │       32       │ 33  │       34       │       35       │ 36  │       37       │       38       │       39       │       40       │ 41  │       42       │       43       │ 44  │       45       │       46       │ 47  │       48       │       49       │       50       │       51       │ 52  │       53       │       54       │ 55  │       56        │       57        │ 58  │       59        │       60        │       61        │       62        │ 63  │       64        │       65        │ 66  │       67        │       68        │ 69  │   

In [56]:
pt_mitiq = generate_pauli_twirl_variants(test, num_circuits=100)

# print(pt_mitiq)
print(len(pt_mitiq))

100


We already can see the compilation time here is substantially faster. 

In [57]:
ankaa = BraketProvider().get_backend("Ankaa-3")

pt_mitiq_verb = to_braket(pt_mitiq, optimization_level=1, target = ankaa.target)

pt_para_circ_verb = to_braket(pt_para_circ, optimization_level=1, target = ankaa.target)


/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:853: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn("The Qiskit circuit contains barrier instructions that are ignored.")
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:903: UserWarning: Device does not support global phase; global phase of 2.3561944901930154 will not be included in Braket circuit
  warnings.warn(
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:903: UserWarning: Device does not support global phase; global phase of 0.78539816339807 will not be included in Braket circuit
  warnings.warn(
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:903: UserWarning: Device does not support global phase; g

In [53]:
pt_params[0]

{'i_0_q0_x': 0.0,
 'i_0_q0_z': 3.141592653589793,
 'i_0_q1_x': 3.141592653589793,
 'i_0_q1_z': 0.0,
 'o_0_q0_x': 0.0,
 'o_0_q0_z': 3.141592653589793,
 'o_0_q1_x': 3.141592653589793,
 'o_0_q1_z': 0.0,
 'i_1_q0_x': 3.141592653589793,
 'i_1_q0_z': 0.0,
 'i_1_q1_x': 0.0,
 'i_1_q1_z': 0.0,
 'o_1_q0_x': 3.141592653589793,
 'o_1_q0_z': 0.0,
 'o_1_q1_x': 3.141592653589793,
 'o_1_q1_z': 0.0,
 'i_2_q1_x': 0.0,
 'i_2_q1_z': 3.141592653589793,
 'i_2_q0_x': 0.0,
 'i_2_q0_z': 3.141592653589793,
 'o_2_q1_x': 0.0,
 'o_2_q1_z': 0.0,
 'o_2_q0_x': 0.0,
 'o_2_q0_z': 3.141592653589793,
 'i_3_q0_x': 3.141592653589793,
 'i_3_q0_z': 0.0,
 'i_3_q1_x': 0.0,
 'i_3_q1_z': 3.141592653589793,
 'o_3_q0_x': 3.141592653589793,
 'o_3_q0_z': 3.141592653589793,
 'o_3_q1_x': 3.141592653589793,
 'o_3_q1_z': 3.141592653589793,
 'i_4_q1_x': 0.0,
 'i_4_q1_z': 3.141592653589793,
 'i_4_q0_x': 0.0,
 'i_4_q0_z': 3.141592653589793,
 'o_4_q1_x': 0.0,
 'o_4_q1_z': 0.0,
 'o_4_q0_x': 0.0,
 'o_4_q0_z': 3.141592653589793,
 'i_5_q0_x': 3

In [58]:
print(pt_para_circ_verb)

T  : │        0        │      1      │                   2                   │                   3                   │     4      │         5          │         6          │      7      │      8      │      9      │   10    │     11     │     12     │     13      │   14    │     15      │     16      │                  17                   │     18     │     19     │         20         │     21     │     22     │                  23                   │     24     │     25     │         26         │     27      │     28      │     29      │   30    │     31     │     32     │     33      │   34    │     35      │     36      │                  37                   │     38     │     39     │         40         │     41      │                  42                   │     43     │     44     │         45         │     46      │     47      │     48      │   49    │     50     │     51     │     52      │   53    │     54      │     55      │                  56                   │     57  

parameters = 

In [ ]:
import time

ps_1 = ProgramSet(pt_mitiq_verb, 
                  shots_per_executable=10)

ps_2 = ProgramSet(
    CircuitBinding(circuit = pt_para_circ_verb, input_sets = pt_params), 
    shots_per_executable=10)

ps_3 = ProgramSet.zip([pt_para_circ_verb]*100, input_sets=pt_params,shots_per_executable=10)


In [ ]:
ankaa3 = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3")
device = ankaa3

os.environ["AWS_REGION"] = "us-west-1"
os.environ["BRAKET_ENDPOINT"] = "https://braket-gamma.us-west-1.amazonaws.com"

start_time = time.time()
task1 = device.run(ps_1)
result1 = task1.result()
end_time = time.time()




In [ ]:


start_time2 = time.time()
task2 = device.run(ps_2)
result2 = task1.result()
end_time2 = time.time()


start_time3 = time.time()
task3 = device.run(ps_3)
result3 = task1.result()
end_time3 = time.time()



In [19]:
import pprint 
import datetime
pprint.pprint(metadata.json())

('{"braketSchemaHeader": {"name": '
 '"braket.task_result.program_set_task_metadata", "version": "1"}, "id": '
 '"arn:aws:braket:us-west-1:641737106670:quantum-task/d1123861-e14d-4035-b373-643566557240", '
 '"deviceId": "arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3", '
 '"requestedShots": 1000, "successfulShots": 1000, "programMetadata": '
 '[{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [{}]}, '
 '{"executables": [{}]}, {"executables": [{}]}, {"executables": [

In [24]:
from datetime import datetime

def print_task_timing(task, result, local_start_time=None, local_end_time=None):
    print(f"Task ARN: {task.id}")
    print(f"Task status: {task.state()}")
    
    if hasattr(result, 'task_metadata'):
        metadata = result.task_metadata
        print(f"Created at: {metadata.createdAt}")
        print(f"Ended at: {metadata.endedAt}")
        
        if metadata.createdAt and metadata.endedAt:
            created = datetime.fromisoformat(metadata.createdAt.replace('Z', '+00:00'))
            ended = datetime.fromisoformat(metadata.endedAt.replace('Z', '+00:00'))
            execution_time = ended - created
            print(f"Execution time: {execution_time.total_seconds():.3f} seconds")
    
    if local_start_time and local_end_time:
        print(f"Local execution time: {local_end_time - local_start_time:.3f} seconds")



In [25]:
print_task_timing(task1,result1,start_time,end_time)

Task ARN: arn:aws:braket:us-west-1:641737106670:quantum-task/d1123861-e14d-4035-b373-643566557240
Task status: COMPLETED
Created at: 2025-11-13T18:17:29.389Z
Ended at: 2025-11-13T18:18:39.769Z
Execution time: 70.380 seconds
Local execution time: 113.992 seconds


In [28]:
print(len(ps_2))

qd = LocalSimulator()
test = qd.run(ps_2).result()

1


In [32]:
device

Device('name': Ankaa-3, 'arn': arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3)

In [46]:
start_time2 = time.time()
task2 = device.run(ps_2)
result2 = task2.result()
end_time2 = time.time()

In [45]:
start_time3 = time.time()
task3 = device.run(ps_3)
result3 = task3.result()
end_time3 = time.time()

ClientError: An error occurred (413) when calling the CreateQuantumTask operation: HTTP content length exceeded 10485760 bytes.

In [47]:
print_task_timing(task2,result2,start_time2,end_time2)

Task is in terminal state FAILED; see result for more details


Task ARN: arn:aws:braket:us-west-1:641737106670:quantum-task/793ca62e-fd14-4f36-9655-13f8aa73babc
Task status: FAILED
Local execution time: 42.458 seconds
